## Setup

In [1]:
import pandas as pd
patient_history = pd.read_csv(
    './vaccine reaction/2021VAERSDATA.CSV', sep=',', encoding='ISO-8859-1')
patient_sympt = pd.read_csv(
    './vaccine reaction/2021VAERSSYMPTOMS.CSV', sep=',', encoding='ISO-8859-1')
patient_vac = pd.read_csv(
    './vaccine reaction/2021VAERSVAX.CSV', sep=',', encoding='ISO-8859-1')

C:\Users\zelon\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7,12,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
pd.set_option("display.max_rows", 1000)

In [3]:
patient_history.tail()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
34116,1113917,03/19/2021,FL,88.0,87.0,NaN,F,NaN,throat closure three weeks after second dose; ...,NaN,...,NaN,Medical History/Concurrent Conditions: Aortic ...,NaN,"USMODERNATX, INC.MOD20210",2,03/12/2021,NaN,NaN,Y,NaN
34117,1113920,03/19/2021,PA,83.0,NaN,NaN,F,NaN,positive COVID-19 test with no symptoms; Asymp...,NaN,...,NaN,Medical History/Concurrent Conditions: Atrial ...,NaN,USPFIZER INC2021231682,2,03/16/2021,NaN,Y,NaN,NaN
34118,1113963,03/19/2021,MS,59.0,NaN,NaN,F,NaN,HLH (hemophagocyticlymphohistiocytosis); This ...,Y,...,NaN,Medical History/Concurrent Conditions: Type II...,NaN,USPFIZER INC2021257356,2,03/12/2021,NaN,NaN,NaN,NaN
34119,1115045,03/19/2021,AZ,57.0,NaN,NaN,M,NaN,Death; A spontaneous report was received from ...,Y,...,NaN,Medical History/Concurrent Conditions: No adve...,NaN,"USMODERNATX, INC.MOD20210",2,03/15/2021,NaN,NaN,NaN,NaN
34120,1115348,03/19/2021,NC,NaN,85.0,NaN,F,NaN,death; A spontaneous report was received from ...,Y,...,NaN,Comments: List of non-encoded Patient Relevant...,NaN,"USMODERNATX, INC.MOD20210",2,03/12/2021,NaN,NaN,NaN,NaN


In [4]:
patient_sympt.tail()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
48105,1113920,Nasal discomfort,23.1,Night sweats,23.1,Platelet count,23.1,Platelet count decreased,23.1,Pyrexia,23.1
48106,1113920,Rash,23.1,SARS-CoV-2 test,23.1,NaN,NaN,NaN,NaN,NaN,NaN
48107,1113963,Haemophagocytic lymphohistiocytosis,23.1,SARS-CoV-2 test,23.1,NaN,NaN,NaN,NaN,NaN,NaN
48108,1115045,Death,23.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48109,1115348,Death,23.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
patient_vac.tail()

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
34625,1113917,COVID19,MODERNA,01120A,1,OT,NaN,COVID19 (COVID19 (MODERNA))
34626,1113920,COVID19,PFIZER\BIONTECH,Unknown,1,NaN,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
34627,1113963,COVID19,PFIZER\BIONTECH,EL9264,2,NaN,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
34628,1115045,COVID19,MODERNA,NaN,2,OT,NaN,COVID19 (COVID19 (MODERNA))
34629,1115348,COVID19,MODERNA,026A21A,1,OT,LA,COVID19 (COVID19 (MODERNA))


In [6]:
patient_history.columns

Index(['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX',
       'RPT_DATE', 'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT',
       'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE',
       'ONSET_DATE', 'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY',
       'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'SPLTTYPE',
       'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT',
       'ALLERGIES'],
      dtype='object')

In [7]:
patient_sympt.columns

Index(['VAERS_ID', 'SYMPTOM1', 'SYMPTOMVERSION1', 'SYMPTOM2',
       'SYMPTOMVERSION2', 'SYMPTOM3', 'SYMPTOMVERSION3', 'SYMPTOM4',
       'SYMPTOMVERSION4', 'SYMPTOM5', 'SYMPTOMVERSION5'],
      dtype='object')

In [8]:
patient_vac.columns

Index(['VAERS_ID', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES',
       'VAX_ROUTE', 'VAX_SITE', 'VAX_NAME'],
      dtype='object')

### Extract the needed columns from the 3 files and combine them to create one dataframe.
* the vacinne symptoms file contains multiple rows for the same vaers_id.This happens when a patient has more than 5 symptoms and each row only holds 5. We need to combine the data to single row before merging with the with the other files.  

In [9]:
# get the needed columns from patient history file
patient_history = patient_history.loc[:, [
    'VAERS_ID', 'AGE_YRS', 'SEX', 'CUR_ILL', 'HISTORY', 'ALLERGIES']]
# get the needed columns from patient vaccine file
patient_vac = patient_vac.loc[:, ['VAERS_ID', 'VAX_NAME']]
# commbining patient's vaccine symptoms to a single cell
patient_sympt['SYMPTOMS'] = patient_sympt['SYMPTOM1'].fillna('')+',' + patient_sympt['SYMPTOM2'].fillna(
    '')+',' + patient_sympt['SYMPTOM3'].fillna('')+',' + patient_sympt['SYMPTOM4'].fillna('')+',' + patient_sympt['SYMPTOM5'].fillna('')
patient_sympt = patient_sympt.groupby(by='VAERS_ID')['SYMPTOMS'].agg(
    lambda x: ','.join(x)).reset_index()

In [10]:
# mergeing all extracted info above and combine them to a single dataframe

df = patient_history.merge(patient_vac, on='VAERS_ID')

In [11]:
df = pd.merge(df, patient_sympt,  left_on='VAERS_ID', right_on='VAERS_ID')
# remove NaN Value
df = df.fillna(value="")
# prepare y values step1
# make symptom columns into a list
df['SYMPTOMS'] = df['SYMPTOMS'].apply(lambda x: x.split(","))
# and than remove any empty values in the list.
df['SYMPTOMS'] = df['SYMPTOMS'].apply(lambda x: [i for i in x if i])
df = df.drop_duplicates(['VAERS_ID'])

In [12]:
df.head()

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS
0,916600,33.0,F,None,None,Pcn and bee venom,COVID19 (COVID19 (MODERNA)),"[Dysphagia, Epiglottitis]"
1,916601,73.0,F,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,"""Dairy""",COVID19 (COVID19 (MODERNA)),"[Anxiety, Dyspnoea]"
2,916602,23.0,F,None,None,Shellfish,COVID19 (COVID19 (PFIZER-BIONTECH)),"[Chest discomfort, Dysphagia, Pain in extremit..."
3,916603,58.0,F,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom...",COVID19 (COVID19 (MODERNA)),"[Dizziness, Fatigue, Mobility decreased]"
4,916604,47.0,F,Na,,Na,COVID19 (COVID19 (MODERNA)),"[Injection site erythema, Injection site pruri..."


### Data Preprocessing 
* Cleaning the data

In [13]:
def convertAge(string_age):
    if string_age == "":
        return "unknown_age"
    if 0 <= float(string_age) <= 2:
        return "0-2 yr"
    if 3 <= float(string_age) <= 12:
        return "3-12 yr"
    if 13 <= float(string_age) <= 19:
        return "13-19 yr"
    if 20 <= float(string_age) <= 39:
        return "20-39 yr"
    if 40 <= float(string_age) <= 59:
        return "40-59 yr"
    return "over_60 yr"


def changeVaxName(vax_name):
    if 'MODERNA' in vax_name:
        return 'MODERNA'
    if 'PFIZER' in vax_name:
        return 'PFIZER'

    return 'JANSSEN'

In [14]:
# remove none coivd related vaccine samples
df = df[df['VAX_NAME'].str.contains(
    '|'.join(['MODERNA', 'PFIZER-BIONTECH'])) & df['VAX_NAME'].isna() == False]
# put age in bucket under 60 or over
df['AGE_CAT'] = df['AGE_YRS'].apply(lambda x: convertAge(x))
# simmplify vaccine names
df['VAX_NAME'] = df['VAX_NAME'].apply(lambda x: changeVaxName(x))

In [15]:
# Create a list of remove words
remove_word_list = ['none', 'na', 'nkda', 'nka', 'unknown', 'medical history/concurrent conditions:', 'no adve...', 'no known allergies', 'no illnesses at time or prior', 'no allergies', 'no adverse event (no reported medical history)', 'comments: list of non-encoded patient relevant...', 'ukn', ' no adverse event (no reported medical history.)',
                    ' no adverse event (no medical history reported)',
                    ' no adverse event (no adverse event was reported.)',
                    ' no adverse event (no medical history reported.)', '',
                    ' no adverse event',
                    ' no adverse event (no patient history reported.)',
                    ' no adverse event (medical history not provided)',
                    ' no adverse event (medical history not reported)',
                    ' no adverse event (no adverse event history)',
                    ' no adverse event (no medical history provided)',
                    ' no adverse event (no reported adverse event)',
                    ' no adverse event (no recorded medical history.)',
                    ' no adverse event (no medical history provided.)',
                    'denies', 'denies any illness at time of vaccition or up ...', 'denies any illnesses', 'patient denies', 'unkown', 'no known', 'comments: list of non-encoded patient relevant...', 'no illnesses at time of vaccition', 'no illness one month prior to or at time of va...']


def remove_words(strings):
    for words in remove_word_list:
        strings = strings.replace(words, '')
    return strings

In [16]:
# make the health_condition text lower case
df['CUR_ILL'] = df['CUR_ILL'].str.lower()
df['HISTORY'] = df['HISTORY'].str.lower()
df['ALLERGIES'] = df['ALLERGIES'].str.lower()
# remove list of words from columns

df['CUR_ILL'] = df['CUR_ILL'].apply(lambda x: remove_words(x))
df['HISTORY'] = df['HISTORY'].apply(lambda x: remove_words(x))
df['ALLERGIES'] = df['ALLERGIES'].apply(lambda x: remove_words(x))

## Extract the columns for training

In [17]:
df_training = df.loc[:, ['AGE_CAT', 'SEX', 'VAX_NAME']]
# combining patients' current illness , medical history and, allergies and store in the "health condition column"
df['HEALTH_CONDITION'] = df['CUR_ILL']+"," + df['HISTORY']+","+df['ALLERGIES']


#### The health_conditons column contains string descriptions of each patient's medical history, allergy, and current illness. We need to extract meaningful key words from them and try to find associations between patients' health conditons and their Vaccine side affects   

In [18]:
import nltk
import itertools
from string import punctuation
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
df.head()

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT,HEALTH_CONDITION
0,916600,33.0,F,,,pcn and bee venom,MODERNA,"[Dysphagia, Epiglottitis]",20-39 yr,",,pcn and bee venom"
1,916601,73.0,F,patient residing at nursing facility. see pati...,patient residing at nursing facility. see pati...,"""dairy""",MODERNA,"[Anxiety, Dyspnoea]",over_60 yr,patient residing at nursing facility. see pati...
2,916602,23.0,F,,,shellfish,PFIZER,"[Chest discomfort, Dysphagia, Pain in extremit...",20-39 yr,",,shellfish"
3,916603,58.0,F,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","diclofec, novacaine, lidocaine, pickles, tomat...",MODERNA,"[Dizziness, Fatigue, Mobility decreased]",40-59 yr,"kidney infection,diverticulitis, mitral valve ..."
4,916604,47.0,F,,,,MODERNA,"[Injection site erythema, Injection site pruri...",40-59 yr,",,"


In [20]:
def remove_words_containning_numbers(word):
    if word.isalpha():
        return word
    return ""

#### tokenize the strings in health_condition column. The column contains descriptions of each patient's medical history, allergy    

In [21]:
# tokenize df_training['HEALTH_CONDITION']
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: nltk.word_tokenize(x))
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

16361

In [22]:
# remove any punctuations from words
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [''.join(c if c not in punctuation else " " for c in s) for s in x])
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

16076

In [23]:
# remove any words mix with number
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [remove_words_containning_numbers(w) for w in x])
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

10565

In [24]:
# remove any empty words
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [string for string in x if string != ""])
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

10564

In [25]:
# remove stop words
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [words for words in x if words not in stopwords])
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
before_remove_noun = allwords
len(allwords)

10433

In [26]:
# extract only nouns form the Health_conditon column.
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [words for words, pos in nltk.pos_tag(x) if pos == "NN" or pos == 'NNS'])

## auto correct spelling errors (run paralle), since it takes a long time to run, I saved the data processed dataframe
import multiprocessing
from autocorrect import Speller
import time
spell = Speller(lang='en')


import mypack.correc as co
import numpy as np
new_dff=df.loc[:,'HEALTH_CONDITION']
new_dff
start_time = time.time()
pool = multiprocessing.Pool(multiprocessing.cpu_count())

new_dff=new_dff.apply(lambda x :  pool.map(co.correction, x ))
   
print("--- %s seconds ---" % (time.time() - start_time))

df['HEALTH_CONDITION']=new_dff

In [27]:
#print([word for word in before_remove_noun if word not in ans])

## CORRECT WORDS MANUALLY

from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

word_grouping=ans.copy()

dic_word=[]
for a in word_grouping:
    
    #print(word_grouping[a])
    
    group=[]
    for b in word_grouping:
        
        if similar(a, b)> 0.80:
            
            group.append (b)
            word_grouping.remove(b)
    dic_word.append(group)
   # word_grouping.remove(word_grouping[a])


print(dic_word)

len(dic_word)

from nltk.corpus import wordnet
def check_isword(x):
    if not wordnet.synsets(x):
        return False
    return True
    

from nltk.corpus import wordnet

new_dic=[]
counter=0
for wordlist in dic_word:
    boo = False
    for word in wordlist:
        if check_isword(word)==True:
            new_dic.append({word:wordlist})
            boo = True
            break
    if boo == False: 
        new_dic.append({counter: wordlist})
        counter = counter + 1

print(new_dic)

temp =[]
for item in new_dic:
    for dic, value in item.items() :
        if type(dic)==str and len(value)>1:
            
            temp.append(item)

temp

### manually checked the simmilar words and clean the data 

In [28]:
simmilar_words = [{'abnormal': ['abnormal', 'abnormality']}, {'acetaminophen': ['acetaminophen', 'acetiminophen']}, {'acyclovir': ['acyclovir', 'alcyclovir']}, {'add': ['add', 'adds']}, {'adenomyosis': ['adenomyosis', 'adneomyosis']}, {'affect': ['affect', 'effects']}, {'agranulocytosis': ['agranulocytosis', 'granulomatosis']}, {'acid': ['acid', 'aid', 'aids']}, {'alcohol': ['alcohol', 'alcohols']}, {'allergy': ['allergy', 'allergies']}, {'alzheimers': ['alheimers', 'alxheimer', 'alzheimers']}, {'amitriptyline': ['amatriptoline', 'amitriptyline']}, {'amoxicillin': ['amoxacillin', 'amoxapine', 'amoxcillin', 'amoxiccillin', 'amoxicillan', 'amoxicillen', 'amoxicilliin', 'amoxicillin', 'amoxillion', 'amoxocin', 'amoxycillan', 'amoxycillian', 'amoxycillin']}, {'abnormalities': ['abnormalities', 'anomalies']}, {'anticoagulant': ['anticoagulant', 'anticoagulation']}, {'ant': ['ant', 'ants']}, {'azithromycin': ['arithimycin', 'arrhthromycin', 'arrythromycin', 'arythomycin', 'azithromycin', 'azithromyocin', 'azythromycin', 'zithromycin']}, {'erthromycin': ['erthromycin', 'erythromycin']}, {'arteriosclerosis': ['arteriosclerosis', 'atherosclerosis', 'otosclerosis']}, {'lovastatin': ['atorvastat', 'atovastatin', 'lovastatin', 'rovastatin']}, {'avocado': ['avocado', 'avocados']}, {'awake': ['awake', 'wake']}, {'bacitracin': ['bacitracin', 'bacitrian', 'bactiracin']}, {'ban': ['ban', 'bans']}, {'bee': ['bee', 'bees']}, {'behavior': ['behavior', 'behaviors']}, {'benzodiazepines': ['benzodiazapenes', 'benzodiazepines']}, {'berries': ['berries', 'bluberries']}, {'breast': ['breast', 'breasts']}, {'bronchiolitis': ['brochitis', 'bronchiolitis']}, {'ibuprofen': ['buprofen', 'ibuprofen']}, {'bypass': ['bypass', 'passes']}, {'calculation': ['calculation', 'circulation', 'coagulation']}, {'cauliflower': ['cauliflower', 'rawcauliflower']}, {'cephalexin': ['cefalaxin', 'cephalexin']}, {'cefuroxime': ['cefuroxime', 'cefuroxme']}, {'center': ['center', 'enter', 'enters']}, {'cephalosporin': ['cefalosporins', 'celphlasporins', 'cephalasporins', 'cephalosporin', 'cephalosporis', 'cephlasporins']}, {'chlorhexidine': ['chlohexidine', 'chlorhexidine', 'cholerahexadine', 'cholorehexadine']}, {'cholecystitis': ['chocystitis', 'cholecystitis']}, {'cholesterol': ['cholesterol', 'clolestroal', 'colorestrol', 'hypercholesterol']}, {'ciprofloxacin': ['ciproflaxen', 'ciprofloxacin']}, {'lincomycin': ['clindamiacin', 'clindomyacin', 'clyndomycin', 'lincomycin']}, {'clinoril': ['clinoril', 'linsopril']}, {'lorazepam': ['clozepam', 'lorazepam']}, {'color': ['color', 'colors']}, {'complain': ['complain', 'complaints']}, {'compassion': ['compassion', 'compression', 'impression']}, {'concentrate': ['concentrate', 'contraction']}, {'condition': ['condition', 'conditioning']}, {'conjunctivitis': ['conjunctivitis', 'rhinoconjunctivitis']}, {'oropharyngeal': ['oropharyngeal', 'sopharyngeal']}, {'cyclobenzaprine': ['cyclobenzaprine', 'cyclobezprine']}, {'hysteroscopy': ['cystoscopy', 'hysteroscopy']}, {'deaf': ['deaf', 'def']}, {'doxycycline': ['doxicillin', 'doxicycline', 'doxocyclin', 'doxycline', 'doxycyclin', 'doxycycline', 'doxycyclineusea', 'doxycylcine', 'doxycyline', 'doxylamine']}, {'deficiency': ['deficiency', 'deficit']}, {'demerol': ['demarol', 'demerol']}, {'dependence': ['dependence', 'dependent']}, {'dexamethasone': ['betamethasone', 'deamethasone', 'dexamethasone']}, {'diabetic': ['diabetic', 'prediabetic']}, {'diagnoses': ['diagnoses', 'diagnostics']}, {'dilaudid': ['dialudid', 'diaudid', 'dilaudid']}, {'diverticulosis': ['diberticulosis', 'divertculosis', 'diverticulosis']}, {'difficult': ['difficult', 'difficulty']}, {'dioxide': ['dioxide', 'oxide']}, {'dis': ['dis', 'disc', 'dish']}, {'lipidemia': ['dislipidemia', 'dyslipemia', 'dysplipdemia', 'lipidemia']}, {'diuresis': ['diuresis', 'duragesis']}, {'draw': ['draw', 'draws']}, {'drop': ['drop', 'drops']}, {'dysphonia': ['dysphonia', 'dysponia']}, {'eastern': ['eastern', 'eaten', 'stern']}, {'egg': ['egg', 'eggs']}, {'neomycin': ['emyacin', 'neomycin']}, {'epicondylitis': ['epicondylitis', 'spondylitis']}, {'esophagitis': ['esofagitis', 'esophagitis', 'esphagitis', 'oesophagitis']}, {'exacerbation': ['exacerbation', 'exascerbation', 'exhacerbation']}, {'eye': ['eye', 'eyes']}, {'fascists': ['fasciatus', 'fascists']}, {'fat': ['fat', 'fats']}, {'feel': ['feel', 'feels']}, {'find': ['find', 'finds']}, {'fir': ['fir', 'fires']}, {'flights': ['flights', 'light', 'lights']}, {'fluorescent': ['fluorescent', 'fluoroscein']}, {'folliculitis': ['foliculitis', 'folliculitis']}, {'form': ['form', 'forms']}, {'fosamax': ['fosamax', 'fosomax']}, {'gastrectomy': ['gastrectomy', 'gastrostomy']}, {'gastroesophageal': ['gastroesophageal', 'gastroesphageal']}, {'gadolinium': ['gadolinium', 'glalinium']}, {'hazelnuts': ['hazeenut', 'hazelnuts']}, {'head': ['head', 'heads']}, {'hemochromatosis': ['hemachromatosis', 'hemochromatosis']}, {'hemodialysis': ['hemodialiysis', 'hemodialysis', 'hemolysis']}, {
    'hypoglycemia': ['hipoglicemia', 'hypoglycemia']}, {'horse': ['horse', 'horses']}, {'hydronephrosis': ['hydonephrosis', 'hydronephrosis']}, {'hydralazine': ['hydralazine', 'hydrazaline']}, {'dehydration': ['dehydration', 'hydration']}, {'hydromorphone': ['hidromorphine', 'hydromorphone']}, {'hydroxy': ['hydro', 'hydroxy']}, {'hyperthyroidism': ['hyopothroidism', 'hyperthyroidism', 'hypohyroidism', 'hypoparathyroidism', 'hyporthyroidism', 'hypothoidism', 'hypothroid', 'hypothryoidism', 'hypothryroidism', 'hypothyriodism', 'hypothyroidism', 'hypothyroiditis', 'hypothyroigism', 'hypthyroidism']}, {'hyperlipidaemia': ['hypelipidemia', 'hyperdyslipidemia', 'hyperlidemia', 'hyperlipademia', 'hyperlipedemia', 'hyperlipidaemia', 'hyperlipidem', 'hyperlipidemia', 'hyperlipidimia', 'hyperlipiemia', 'hypolipidemia', 'hyprlipidemia']}, {'hypercalcemia': ['hpercalcemia', 'hypercalcemia', 'hyperglycemia', 'hyperkalemia']}, {'hypercholesterolemia': ['hyperchloestermia', 'hypercholeroltemia', 'hypercholesteralemia', 'hypercholestermia', 'hypercholesterolemia', 'hypercholesterolism', 'hypercholsterolemia', 'hyperholesteremia', 'hypersholesteremia', 'hypocholesteremia', 'ypercholesteremia', 'hypercolestoramia']}, {'hyperhidrosis': ['hyperhidrosis', 'hyperhydrosis', 'hyperthyrodism']}, {'hypermetropia': ['hypermetropia', 'hyperopia']}, {'hyperparathyroidism': ['hyperparathyroidism', 'hypertyroidism', 'hypoparathyroidim']}, {'hypertension': ['hypertension', 'prehypertension']}, {'hypovolemia': ['hypocholemia', 'hypovolemia']}, {'hypoglycemic': ['hypoglycemic', 'hypooglycemia']}, {'hypokalemia': ['hyokalemia', 'hypokalemia']}, {'hypoventilation': ['hypoventilation', 'ventilation']}, {'inflammation': ['inflammation', 'inflammatory']}, {'inhibition': ['inhibition', 'inhibitors']}, {'laminectomy': ['hemilaminectomy', 'laminectomy']}, {'laryngitis': ['laryngitis', 'pharyngitis']}, {'leaves': ['leaves', 'level']}, {'lip': ['lip', 'lips']}, {'list': ['list', 'lists']}, {'lymphedema': ['lymphadem', 'lymphedema']}, {'lymphoid': ['lymphnode', 'lymphoid']}, {'macrodantin': ['macrodantin', 'microdantin']}, {'macrocytosis': ['macrocytosis', 'mastocytosis', 'stomatocytosis']}, {'menopausal': ['menopausal', 'perimenopausal', 'postmenopausal']}, {'menorrhagia': ['menometrorrhagia', 'menorrhagia']}, {'metabolic': ['hypometabolism', 'metabolic']}, {'albuminuria': ['albuminuria', 'microalbuminuria']}, {'minocycline': ['minicycline', 'minocycline']}, {'monistat': ['monistat', 'monostat']}, {'myocarditis': ['myocarditis', 'myopericarditis']}, {'neosporin': ['neospin', 'neosporin']}, {'nephritis': ['nephritis', 'neuritis', 'polynephritis']}, {'neuron': ['neuro', 'neuron', 'neurons']}, {'neurofibromatosis': ['fibromatosis', 'neurofibromatosis']}, {'sarcoidosis': ['neurosarcoidosis', 'sarcoidosis']}, {'night': ['night', 'nights']}, {'nitrofuran': ['nitrofuran', 'nitrofurantoin']}, {'melanoma': ['melanoma', 'nonmelanoma']}, {'nut': ['nut', 'nuts']}, {'occasion': ['occasion', 'occlusion']}, {'orbital': ['orbital', 'orita']}, {'ephedrine': ['ephedrine', 'orphedrine']}, {'osteopetrosis': ['oesteoporosis', 'osteonecrosis', 'osteopetrosis', 'osteoporosis', 'osteporosis', 'ostprosis']}, {'palpitations': ['palpatation', 'palpitatioins', 'palpitations']}, {'parkinson': ['parkinson', 'parkinsonisms']}, {'pepper': ['pepper', 'peppers']}, {'laryngeal': ['laryngeal', 'pharyngeal']}, {'phenergan': ['phenergan', 'phergergan']}, {'phenobarbitals': ['phenobarb', 'phenobarbitals']}, {'pig': ['pig', 'pigs']}, {'pistachio': ['pistachio', 'pistschios']}, {'place': ['place', 'places']}, {'plan': ['plan', 'plans']}, {'polyarteritis': ['polyarteritis', 'polyarthritis']}, {'preservatives': ['perservatives', 'preservatives']}, {'propoxyphene': ['propoxyphene', 'propyphene']}, {'pseudoephedrine': ['psedophedrine', 'pseudoephedrine', 'pseudophendirine']}, {'quadriplegia': ['quadraplegic', 'quadriplegia']}, {'question': ['question', 'questions']}, {'benzalclonium': ['benzalcloniumbenzalkonium', 'benzene', 'benzoate', 'benzocaine', 'benzodiazepine', 'benzodiazepines', 'benzoid', 'benzoly', 'benzoperoxide', 'benzoyl']}, {'bronchiectasis': ['bronchial', 'bronchiectasis', 'bronchiolitis', 'bronchitis', 'bronchopneumonia', 'bronchospasm']}, {'cranberries': ['cranberries', 'raspberries']}, {'result': ['result', 'results']}, {'rheumatoid': ['rheumatiod', 'rheumatoid']}, {'run': ['run', 'runs']}, {'salicylate': ['salicylate', 'salicylicates']}, {'thrombocytopemia': ['thrombocytopemia', 'thrombocytopenia', 'thrombocytosis', 'thromboembolism', 'thrombophilia', 'thrombophlebitis', 'thrombus']}, {'ischemia': ['ischemia', 'schema']}, {'shellfish': ['selfish', 'shellfish']}, {'shoot': ['shoot', 'shot']}, {'sign': ['sign', 'signs']}, {'penicillins': ['pencillers', 'peneciline', 'penicillin', 'penillicillins', 'pennicellins']}, {'sinusitis': ['sinnusitis', 'sinusitis', 'tinnitis']}, {'sleep': ['sleep', 'sleeps']}, {'smoke': ['smoke', 'smokers']}]

In [29]:
simmilar_words2 = [{'hypogammaglobulinemia': ['ypogammaglobuliemia', 'hypogammaglobulinemia', 'ypogammaglobulonemia']}, {'spironolactone': ['spironolactone', 'sprionolactone']}, {'spondylolisthesis': ['spondylolisthesis', 'spondylolysis', 'spondylosis', 'spondylolithesis', 'spondylopathies']}, {'stay': ['stay', 'stays']}, {'sulfanilamide': ['sulfailamide', 'sulfanilamide']}, {'sulfide': ['sulfide', 'sulfomides']}, {'supplement': ['supplement', 'supplements']}, {'tachycardia': ['tacacardia', 'tachcardia', 'tachycardia']}, {'tetracyclines': ['tetracicline', 'tetracyclines']}, {'thioridazine': ['thioridazine', 'thorazine']}, {'thrombocytosis': ['thrombocytosis', 'thrombosis']}, {'issues': ['issues', 'tissues']}, {'titan': ['titan', 'titans']}, {'tobramycin': ['tobdramycin', 'tobramycin']}, {'tolectin': ['tolectin', 'tolmetin']}, {'toxicity': ['ototoxicity', 'toxicity']}, {'transplant': ['transplant', 'transplants']}, {'tube': ['tube', 'tubes']}, {'onions': ['onions', 'union']}, {'urticaria': ['uritcaria', 'urticaria', 'uticaria']}, {'value': ['value', 'valves']}, {'vancomycin': ['vancomycin', 'vancomyocin']}, {'atrioventricular': ['atrioventricular', 'ventricular']}, {'visit': ['visit', 'visits']}, {'week': ['week', 'weeks']}, {'well': ['well', 'wells']}, {'abnormal': ['abnormal', 'abnormality']}, {'acetaminophen': ['acetaminophen', 'acetaminophine', 'acetate', 'acetominiphen']}, {'abscess': ['abscess', 'abscessus']}, {'absence': ['absence', 'absent']}, {'adenocarcinoma': ['adenocarcinoma', 'adenoma', 'adenomyosis', 'adenosine']}, {'adhesion': ['adhesion', 'adhesive']}, {'admin': ['admin', 'administration']}, {'admit': ['admit', 'admits']}, {'agenesis': ['agenesis', 'agenisis']}, {'albumin': ['albumin', 'albuminuria']}, {'alergies': ['alergies', 'allegergies', 'allegeries', 'allegies', 'allegra', 'allergans', 'allergen', 'allergic', 'allergiers', 'allergiex', 'allergist', 'allergy', 'alleriges']}, {'aluminium': ['aluminium', 'aluminum']}, {'alzheimer': ['alzheimer', 'alzheimers']}, {'amblyopia': ['amblyopia', 'ambulance', 'ambulatory']}, {'aminoglycosides': ['aminoglycosides', 'aminooglycosides']}, {'amiodarone': ['amiodarone', 'amioderone']}, {'amitriptyline': ['amitriptyline', 'amitryptiline']}, {'amlodipine': ['amlodipine', 'amlodopine']}, {'amox': ['amox', 'amoxi', 'amoxicillin', 'amoxil']}, {'ampicillin': ['ampicillin', 'ampicillinhives']}, {'amputaion': ['amputaion', 'amputation']}, {'anemia': ['anemia', 'anemic', 'anemica']}, {'anesthesia': ['anesthesia', 'anesthessia', 'anesthetic']}, {'aneurism': ['aneurism', 'aneursym', 'aneurysm']}, {'angeoedema': ['angeoedema', 'angio', 'angiodema', 'angiodysplasia', 'angioedema', 'angiogram', 'angiomyolipoma', 'angiopathy', 'angioplasty', 'angiotensin', 'angloidosis']}, {'antibiotic': ['antibiotic', 'antibiotoics', 'antibitotics', 'antibody', 'antiboitics']}, {'anticoag': ['anticoag', 'anticoagulant']}, {'anuerysm': ['anuerysm', 'anurism', 'anursym']}, {'anxiety': ['anxiety', 'anxity']}, {'aphagia': ['aphagia', 'aphalaxic', 'aphalaxis', 'aphalxis', 'aphalyxis', 'aphasia']}, {'aphylaxes': ['aphylaxes', 'aphylaxis', 'aphylaxtis']}, {'apnea': ['apnea', 'apneia', 'apnoea']}, {'appendectomy': ['appendectomy', 'appendiceal', 'appendicitis', 'appendix']}, {'arrhythmia': ['arrhythmia', 'arrythmia']}, {'arteriole': ['arteriole', 'arteriosclerosis', 'arteritis', 'artery', 'arthirits', 'arthitis', 'arthralgia', 'arthria', 'arthrities', 'arthritis', 'arthritus', 'arthrodesis', 'arthrofibrosis', 'arthropathy', 'arthroplasty', 'arthropod', 'arthroscopy', 'arthrrities', 'artritis']}, {'asbestos': ['asbestos', 'asbestosis']}, {'aasthma': ['aasthma', 'ashma', 'ashtma', 'asma', 'asthma', 'asthmatic', 'asthmaticus', 'athsma']}, {'asparagus': ['asparagus', 'aspartame', 'aspartane', 'astio', 'astomosis']}, {'aspercream': ['aspercream', 'asperger', 'aspergers', 'aspertane', 'aspertein', 'aspirin', 'aspirne', 'atelol', 'atenolol', 'atenolot']}, {'asplenia': ['asplenia', 'asplenism', 'augementin', 'augmentation', 'augmenten', 'augmentin', 'augmenton']}, {'avacado': ['avacado', 'avacodo', 'avacodos', 'avocado', 'avocadoes']}, {'avalox': ['avalox', 'avelox', 'azithromiacine', 'azithromycin', 'azithromyecin', 'azythramycin', 'bacteremia', 'bacteria', 'bacterium', 'bactrain', 'bactria', 'bactrim', 'bactrin', 'bactrium', 'bactriun', 'bactroban', 'bactruim', 'bactrum']}, {'barret': ['barret', 'barrets', 'barrett', 'barretts']}, {'benzalclonium': ['benzalclonium', 'benzalkonium']}, {'benzo': ['benzo', 'benzodiazepine', 'benzoin', 'benzoine', 'benzol', 'benzotate', 'benzote']}, {'bezapril': ['bezapril', 'bezepril']}, {'biacin': ['biacin', 'biactin']}, {'biaxcin': ['biaxcin', 'biaxin']}, {'bicep': ['bicep', 'biceps']}, {'bifid': ['bifid', 'bifida']}, {'bleed': ['bleed', 'bleeding', 'bleeds']}, {'blind': ['blind', 'blindness']}, {'bronchiectasis': ['bronchiectasis', 'bronchiolitis', 'bronchus']}, {'bupropion': ['bupropion', 'buproprion']}, {'butyl': ['butyl', 'butylcarbamate']}, {'candida': [
    'candida', 'candidate', 'candidiasis']}, {'cantalope': ['cantalope', 'cantaloupe', 'canteloupe']}, {'capsaicin': ['capsaicin', 'capsasin']}, {'carbamate': ['carbamate', 'carbamaze', 'carbamazepine', 'carbamazepone']}, {'carcinoid': ['carcinoid', 'carcinoma', 'carcinosarcoma']}, {'cardia': ['cardia', 'cardiac', 'cardio', 'cardiologist', 'cardiology', 'cardiomediastil', 'cardiomegaly', 'cardiomyopathy', 'cardiorel', 'cardiovascular', 'cardioversion', 'cardioversions']}, {'cefaclor': ['cefaclor', 'cefador', 'cefadroxil', 'cefalexin', 'cefazolin']}, {'cefdenir': ['cefdenir', 'cefdiner', 'cefdinir', 'cefdnir', 'cefnidir']}, {'cefepime': ['cefepime', 'cefipime']}, {'celluitis': ['celluitis', 'cellulitis']}, {'cephalaxin': ['cephalaxin', 'cephalexin', 'cephalon', 'cephalosporin', 'cephasporins']}, {'cephlex': ['cephlex', 'cephlexin', 'ceplalaxin']}, {'cerebri': ['cerebri', 'cerebritis', 'cerebro', 'cerebrovasc', 'cerebrovascular', 'cerebyx']}, {'cerumen': ['cerumen', 'cerumenex']}, {'chemo': ['chemo', 'chemoradiation', 'chemotherapy', 'chemothrapy']}, {'chlor': ['chlor', 'chlorahexidine', 'chloramphenicol', 'chloraprep', 'chlorascrub', 'chlordiazepoxide', 'chloresterol', 'chlorhexidine', 'chloride', 'chlorine', 'chlormycetin', 'chloroprep', 'chloroquine', 'chlorothiazide', 'chloroxylenol', 'chlorpheniramine', 'chlorpromazine', 'chlorthalidone']}, {'chocholate': ['chocholate', 'chocolate']}, {'cholecystectomy': ['cholecystectomy', 'cholecystitis', 'choledocholithiasis', 'cholelithiasis']}, {'chloesterol': ['chloesterol', 'cholersterol', 'cholesteral', 'cholesterol', 'cholesterolemia', 'cholesterolm', 'cholestrol', 'cholestyramine']}, {'cillin': ['cillin', 'cillins']}, {'ciprodex': ['ciprodex', 'ciproflaxin', 'ciprofloxaci', 'ciprofloxacin', 'ciprofloxin']}, {'clarithomycin': ['clarithomycin', 'clarithromycin']}, {'clindamayacin': ['clindamayacin', 'clindamyacin', 'clindamyaxin', 'clindamycin', 'clindamycinrash', 'clindamysin', 'clindaymyacin', 'clindomycin']}, {'colities': ['colities', 'colitis', 'colits']}, {'colonoscopy': ['colonoscopy', 'colonscopy']}, {'compazene': ['compazene', 'compazine', 'compezine']}, {'congestion': ['congestion', 'congestive']}, {'conjuctivits': ['conjuctivits', 'conjunctivitis', 'conjuntivitis']}, {'constiipation': ['constiipation', 'constipation']}, {'costichondritis': ['costichondritis', 'costochondritis', 'costochronitis']}, {'cyclobenzaprine': ['cyclobenzaprine', 'cyclobenzoprine', 'cyclodrine', 'cyclosporin', 'cyclosporine']}, {'dabigatran': ['dabigatran', 'dabigitran']}, {'defibrilator': ['defibrilator', 'defibrillator', 'defibulator']}, {'deficency': ['deficency', 'deficience', 'deficiency', 'deficient', 'deficit']}, {'demand': ['demand', 'demenita', 'dementia', 'dementio']}, {'depacote': ['depacote', 'depakote']}, {'depatale': ['depatale', 'dependence', 'dependency']}, {'dermatofibroma': ['dermatofibroma', 'dermatographia', 'dermatographism', 'dermatologist', 'dermatomyocitis', 'dermatomyositis', 'dermatophytosis', 'dermatoymositis']}, {'diabertes': ['diabertes', 'diabet', 'diabetes', 'diabetic', 'diabeties', 'diabetis', 'diaibetes']}, {'diahhrea': ['diahhrea', 'diahrea', 'diarreha', 'diarrhea', 'diarrhoea', 'dibetes']}, {'dicloxacilliam': ['dicloxacilliam', 'dicloxacillin', 'dicloxicillin']}, {'diphenhyd': ['diphenhyd', 'diphenhydramine']}, {'diverculitis': ['diverculitis', 'diversion', 'diverticulas', 'diverticulitis', 'diverticulosis', 'diverticululitis', 'diverticulum', 'divertivulitis']}, {'dizziness': ['dizziness', 'dizzy']}, {'dupuytrans': ['dupuytrans', 'dupuytren', 'dupuytrens']}, {'duricef': ['duricef', 'duriceff']}, {'dustmite': ['dustmite', 'dustmites']}, {'dyspahgia': ['dyspahgia', 'dyspepsia', 'dysplasia']}, {'dysphagia': ['dysphagia', 'dysphonia', 'dysphoria', 'dysphoric']}, {'dysrhythmia': ['dysrhythmia', 'dysthymia']}, {'emphezema': ['emphezema', 'emphysema']}, {'encephalitis': ['encephalitis', 'encephalopathy']}, {'endocin': ['endocin', 'endocrine', 'endocrinologist', 'endocrinology']}, {'endometeriosis': ['endometeriosis', 'endometriosis', 'endometropsis', 'endometrosis', 'endrometriosis']}, {'enviormental': ['enviormental', 'environment']}, {'eosinophil': ['eosinophil', 'eosinophilia']}, {'epicondyle': ['epicondyle', 'epicondylitis']}, {'erythema': ['erythema', 'erythematosis', 'erythematosus']}, {'erythmycin': ['erythmycin', 'erythomycin', 'erythritol', 'erythrmycin']}, {'erythromelalgia': ['erythromelalgia', 'erythromiacin', 'erythromicin', 'erythromyacin', 'erythropoietin', 'erytrhomycin']}, {'esophageal': ['esophageal', 'esophagectomy', 'esophagitis', 'esophagus', 'esophogus', 'esphogael']}, {'ethyl': ['ethyl', 'ethylenediamine']}, {'fatigue': ['fatigue', 'fatique']}, {'fasciatis': ['fasciatis', 'fasciculation', 'fasciitis', 'fascisitis', 'fascitis']}, {'fentanol': ['fentanol', 'fentanyl', 'fentayl', 'fentyl', 'fentynol']}, {'fibirllation': ['fibirllation', 'fibrilation', 'fibrilla', 'fibrillation']}, {'fibroma': ['fibroma', 'fibromialyia', 'fibromyalgia', 'fibromylagia']}]

In [30]:
simmilar_words3 = [{'fibrosing': ['fibrosing', 'fibrosis']}, {'flaygel': ['flaygel', 'flaygyll']}, {'flexaril': ['flexaril', 'flexeril']}, {'floroquinolines': ['floroquinolines', 'floroquinolins']}, {'fluorescein': ['fluorescein', 'fluorescens', 'fluorescent']}, {'fluoro': ['fluoro', 'fluoroquinolone', 'fluoroquinolones', 'fluorouracil']}, {'fluoxetine': ['fluoxetine', 'fluoxitine']}, {'fluticasone': ['fluticasone', 'fluticazolone']}, {'gabapentin': ['gabapentin', 'gabapetin']}, {'gabipentin': ['gabipentin', 'gabipenton', 'gabupentin']}, {'gadolineum': ['gadolineum', 'gadolinium']}, {'gastrocnemius': ['gastrocnemius', 'gastroeneteritis', 'gastroenteritis']}, {'gastroesophageal': ['gastroesophageal', 'gastroesophegeal', 'gastrographin', 'gastrointestil', 'gastroparesis', 'gastroparisis', 'gastrophoresis']}, {'gluten': ['gluten', 'glutete', 'gluteus']}, {'glycol': ['glycol', 'glycoprotein']}, {'haschimoto': ['haschimoto', 'hashimoto', 'hashimotos', 'hashimottos', 'hashitomotos', 'hashmotos', 'hasimotois']}, {'headache': ['headache', 'headahce', 'headedness']}, {'hemahromacytosis': ['hemahromacytosis', 'hemangioma', 'hemaplegia', 'hemarthrosis', 'hematemesis', 'hematoma', 'hematospermia', 'hematuria']}, {'hemi': ['hemi', 'hemicolectomy', 'hemicrania', 'hemiparesis', 'hemiplegia', 'hemiplegic']}, {'hemochromatosis': ['hemochromatosis', 'hemochromotosis']}, {'hemorrhage': ['hemorrhage', 'hemorrhagic', 'hemorrhoid']}, {'hemoglobin': ['hemoglobin', 'hemoglobinuria']}, {'hemodialyis': ['hemodialyis', 'hemodialysis']}, {'hepatitis': ['hepatitis', 'hepatits']}, {'hoshimoto': ['hoshimoto', 'hoshimotos', 'hosimitos']}, {'hydrochlorathiazide': ['hydrochlorathiazide', 'hydrochloride', 'hydrochlorithiazide', 'hydrochlorothiazide', 'hydrochlorthiazide', 'hydrochodone', 'hydrochorothiazide', 'hydroclorothiazide']}, {'hydrocodine': ['hydrocodine', 'hydrocodone', 'hydrocone', 'hydrocortisone']}, {'hydroxychloroquie': ['hydroxychloroquie', 'hydroxychloroquine', 'hydroxychlorquine', 'hydrozychloroquine']}, {'hyperacuasis': ['hyperacuasis', 'hyperacusis']}, {'hyperaldosternoism': ['hyperaldosternoism', 'hyperaldosteronism']}, {'hyperchloesterolemia': ['hyperchloesterolemia', 'hyperchol', 'hypercholestemia', 'hypercholesteremia', 'hypercholesterolemi', 'hypercholesterolemia', 'hypercholesterolimia', 'hypercholestrolemia']}, {'hyperhomocysteinemia': ['hyperhomocysteinemia', 'hyperhomocystinemia']}, {'hyperlidipdemia': ['hyperlidipdemia', 'hyperlipdemia', 'hyperlipemia', 'hyperlipidaemia', 'hyperlipideia', 'hyperlipidema', 'hyperlipidemis', 'hyperlipiden', 'hyperlipids']}, {'hypermetropia': ['hypermetropia', 'hypermobile', 'hypermobility']}, {'hyperosmoloty': ['hyperosmoloty', 'hyperostosis']}, {'hyperparathyroidism': ['hyperparathyroidism', 'hyperparthyrodism']}, {'hyperpotassemia': ['hyperpotassemia', 'hyperprolactemia', 'hyperprolactinemia']}, {'hypertenision': ['hypertenision', 'hypertennsion', 'hypertension', 'hypertensive', 'hypertenstion', 'hypertention', 'hypertesnion', 'hypertnesion', 'hypetension', 'hytertension']}, {'hypertriclyceridemia': ['hypertriclyceridemia', 'hypertriglycerademia', 'hypertriglyceridaemia', 'hypertriglyceridemia', 'hypogammaglobuliemia', 'hypogammaglobulinemia',
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           'inflamation', 'inflammation', 'inflammatories', 'infmamation']}, {'hypertrophic': ['hypertrophic', 'hypertrophy', 'hypoglycemia', 'hypoglycemic']}, {'hyperurecemia': ['hyperurecemia', 'hyperuricemia', 'hypokalemia', 'hypokalmia']}, {'hypomagnesemia': ['hypomagnesemia', 'hypomagnesia']}, {'hypothridism': ['hypothridism', 'hypothroidism', 'hypothryodism', 'hypothryoid', 'hypothryroid', 'hypothydriodism', 'hypothyrodism', 'hypothyroid', 'hypothyroidisn', 'hypothyroidontism']}, {'hypoxemia': ['hypoxemia', 'hypoxia', 'hypoxic', 'hypoxy', 'leukaemia', 'leukemia', 'leukoaraiosis', 'leukocyte', 'leukocytosis', 'leukopenia']}, {'lisiniopril': ['lisiniopril', 'lisinipril', 'lisinopril', 'lisinopryl', 'lisonipril']}, {'lipedema': ['lipedema', 'lipidemia']}, {'lorazepam': ['lorazepam', 'lorazepan']}, {'lymph': ['lymph', 'lymphadema', 'lymphadenopathy', 'lymphangioleiomyomatosis', 'lymphedema', 'lymphedma', 'lymphoangioleiomymatosis', 'lymphocytosis', 'lymphodema', 'lymphohistiocytosis', 'lymphoid', 'lymphoma']}, {'methylchloroisothiazinoline': ['methylchloroisothiazinoline', 'methyldibromoglutaronitrile']}, {'microademnoma': ['microademnoma', 'microadenoma']}, {'miggraine': ['miggraine', 'migraine', 'migrainosus', 'migrains', 'migrane', 'migranes', 'migrans']}, {'oropharangeal': ['oropharangeal', 'oropharyngeal']}, {'osteo': ['osteo', 'osteoarthritics', 'osteoarthritis', 'osteoarthrosis', 'osteochondritis', 'osteodystrophy', 'osteodystrpphy', 'osteomyelitis', 'osteopenia', 'osteoperosis', 'osteopetrosis', 'osteoporasis', 'osteoporisis', 'osteoporsis', 'osteosrthritis', 'osteroarthritis', 'osteroporosis']}, {'pancolitis': ['pancolitis', 'pancreas', 'pancreatic', 'pancreatitis']}, {'pencilin': ['pencilin', 'pencillin', 'pencillins', 'penecilin', 'penecillan', 'penecillin', 'penicciclin', 'peniccilin', 'peniccillin', 'penicellin', 'peniciilin', 'penicilin', 'penicillan', 'penicillans', 'penicillen', 'penicillian', 'penicilliin', 'penicillin', 'penicilline', 'penicllin', 'penincillin', 'pennecillin', 'pennicillan', 'pennicillen', 'pennicillin', 'pennicillink', 'pennicillins', 'pennicillun', 'pennicllin']}, {'pharyngoesophageal': ['pharyngoesophageal', 'pharyngoesphageal']}, {'phenegran': ['phenegran', 'phenegren', 'phenegrin', 'phenelzine', 'phenergan', 'phenergin', 'phenethyl']}, {'phenobarbital': ['phenobarbital', 'phenobarbitol']}, {'phenothiazine': ['phenothiazine', 'phenothyazines', 'phenotype']}, {'pnemonia': ['pnemonia', 'pneumomia', 'pneumonia', 'pneumonitis', 'pneumothoracic', 'pneumothorax', 'pneumovax', 'pnuemonia']}, {'prednison': ['prednison', 'prednisone', 'prednizone', 'prenisone', 'prenizone']}, {'pseudephedrine': ['pseudephedrine', 'pseudobrofimide', 'pseudobulbar']}, {'pseudoephedrine': ['pseudoephedrine', 'pseudoephidrine']}, {'pseudoseizure': ['pseudoseizure', 'pseudoseizures']}, {'thiermosol': ['thiermosol', 'thimerasol', 'thimerisol', 'thimerosal', 'thimerosol', 'thimersol']}, {'thyroidectomy': ['thyroidectomy', 'thyroidemctomy', 'thyroidism', 'thyroiditis', 'thyroidits', 'thyroidproblem', 'thyroxine']}, {'waldenstrom': ['waldenstrom', 'waldenström', 'waldonstrom']}, {'symptoms': ['symproms', 'symptom', 'symptomatic', 'symptoms', 'symtoms', 'symtpoms']}]

In [31]:
def change_words(word):
    for dic in simmilar_words:
        for key, values in dic.items():
            if word in values:
                return key
    return word


def change_words2(word):
    for dic in simmilar_words2:
        for key, values in dic.items():
            if word in values:
                return key
    return word


def change_words3(word):
    for dic in simmilar_words3:
        for key, values in dic.items():
            if word in values:
                return key
    return word

In [32]:
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [change_words(word) for word in x])
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [change_words2(word) for word in x])
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [change_words3(word) for word in x])

In [33]:
meaningless = ['comment', 'history', 'patient', 'history', 'aaa', 'ab', 'abd', 'abt', 'ac', 'ace', 'acf', 'acid', 'acne', 'acre', 'add', 'adh', 'adm', 'adn', 'ae', 'af', 'afi', 'agar', 'age', 'ahd', 'air', 'aka', 'aki', 'al', 'alan', 'alex', 'alf', 'ali', 'alk', 'all', 'alla', 'alma', 'alot', 'als', 'alt', 'am', 'amd', 'amor', 'amp', 'ams', 'ance', 'ang', 'anl', 'ant', 'any', 'ap', 'apc', 'ape', 'apex', 'app', 'apps', 'apr', 'aps', 'arb', 'arch', 'arcs', 'are', 'area', 'arf', 'aria', 'arid', 'arm', 'arms', 'army', 'arts', 'asa', 'asap', 'asd', 'ash', 'asia', 'ast', 'atb', 'ate', 'atom', 'au', 'aug', 'auto', 'av', 'avg', 'awa', 'az', 'b', 'ba', 'back', 'bag', 'ban', 'band', 'bar', 'bare', 'bark', 'barr', 'bars', 'bas', 'base', 'bav', 'bay', 'bbb', 'bc', 'bcc', 'bed', 'been', 'bg', 'bh', 'bha', 'bhp', 'bi', 'bid', 'bil', 'bio', 'bit', 'bk', 'ble', 'bmi', 'bms', 'bnp', 'bot', 'bp', 'br', 'bs', 'bss', 'bug', 'bun', 'bus', 'but', 'c', 'ca', 'cab', 'cad', 'can', 'cap', 'car', 'cas', 'cat', 'cbc', 'cbs', 'cc', 'cce', 'ccm', 'cd', 'cf', 'ch', 'chg', 'chi', 'chr', 'ci', 'cic', 'cid', 'cin', 'cir', 'cis', 'ciu', 'ck', 'cko', 'cl', 'cm', 'cmc', 'cmo', 'cmp', 'cms', 'cmt', 'col', 'con', 'cor', 'cow', 'cp', 'cr', 'crc', 'cri', 'cry', 'cs', 'ct', 'cta', 'cts', 'cum', 'cut', 'cv', 'cvs', 'da', 'dad', 'day', 'dbs', 'dcv', 'ddd', 'de', 'dec', 'del', 'des', 'dhe', 'di', 'dia', 'did', 'die', 'dif', 'dip', 'dir', 'dis', 'div', 'dl', 'dm', 'dn', 'dnr', 'doe', 'dog', 'dot', 'dp', 'dr', 'dry', 'ds', 'du', 'duo', 'dur', 'dx', 'dye', 'dyn', 'dz', 'e', 'ear', 'eat', 'eb', 'ed', 'eds', 'ees', 'ef', 'ehr', 'em', 'ems', 'end', 'eng', 'ent', 'epi', 'er', 'erm', 'esp', 'esr', 'et', 'eta', 'etc', 'eth', 'exp', 'f', 'fa', 'fat', 'fe', 'feb', 'fil', 'fir', 'fix', 'fld', 'flu', 'fmf', 'fms', 'fr', 'fur', 'fx', 'g', 'ga', 'gad', 'gag', 'gal', 'gb', 'gbs', 'ge', 'gel', 'gen', 'ger', 'get', 'gi', 'gm', 'gol', 'got', 'gp', 'gt', 'gum', 'gun', 'gut', 'h', 'ha', 'had', 'hay', 'hb', 'hbo', 'hc', 'hd', 'hdl', 'heb', 'hep', 'hf', 'hh', 'hi', 'hip', 'hit', 'hiv', 'hl', 'ho', 'hop', 'how', 'hp', 'hpt', 'hr', 'hrs', 'hrt', 'hsi', 'hsv', 'ht', 'htm', 'hx', 'ia', 'ic', 'ica', 'icd', 'ice', 'icu', 'id', 'ida', 'idd', 'idm', 'idp', 'if', 'ig', 'iga', 'ige', 'igm',
               'ii', 'iii', 'il', 'ill', 'ils', 'im', 'img', 'in', 'inh', 'inj', 'ink', 'int', 'ip', 'iq', 'is', 'it', 'iu', 'iv', 'ivi', 'ivy', 'j', 'jan', 'jaw', 'job', 'jul', 'k', 'ka', 'kd', 'kg', 'kid', 'kts', 'ku', 'l', 'la', 'lab', 'lad', 'lap', 'lb', 'lbs', 'le', 'leg', 'let', 'lip', 'll', 'llt', 'lon', 'lot', 'lp', 'lpg', 'lt', 'ltf', 'lv', 'ma', 'mac', 'mag', 'mai', 'mal', 'man', 'map', 'mar', 'max', 'mc', 'mca', 'mcd', 'mcg', 'mcs', 'md', 'mdd', 'mdr', 'mds', 'mes', 'mg', 'mgm', 'mi', 'mid', 'mil', 'mix', 'ml', 'mla', 'mm', 'mo', 'mod', 'mom', 'mos', 'mpa', 'mph', 'mr', 'ms', 'msa', 'msg', 'mst', 'mus', 'mvc', 'mvp', 'mw', 'n', 'nc', 'nd', 'nda', 'neg', 'nfa', 'nh', 'nhl', 'nid', 'nil', 'nk', 'nka', 'nma', 'nn', 'non', 'nor', 'nos', 'nov', 'npo', 'ns', 'nt', 'nta', 'num', 'nut', 'nya', 'oa', 'oak', 'oat', 'occ', 'oct', 'od', 'oil', 'ol', 'old', 'om', 'one', 'op', 'opa', 'os', 'osa', 'oth', 'ou', 'out', 'ox', 'oxy', 'p', 'pa', 'pab', 'pad', 'pai', 'pak', 'pap', 'pat', 'pay', 'pb', 'pd', 'pda', 'pdf', 'pe', 'pea', 'pec', 'peg', 'pen', 'per', 'pes', 'pet', 'pf', 'pfo', 'pfr', 'ph', 'phx', 'pid', 'pig', 'pit', 'pm', 'pmc', 'pmn', 'pmr', 'pms', 'po', 'poc', 'pog', 'pos', 'pox', 'pp', 'ppi', 'ppm', 'ppv', 'pr', 'prd', 'pre', 'prn', 'pro', 'psa', 'psp', 'pt', 'pts', 'put', 'pvt', 'q', 'qd', 'qt', 'r', 'ra', 'rad', 'rag', 'rah', 'rat', 'ray', 'rca', 'rec', 'red', 'rel', 'ret', 'rev', 'rib', 'rka', 'rle', 'rls', 'rn', 'rod', 'rom', 'row', 'rr', 'rt', 'rue', 'run', 'rv', 'rx', 'ry', 'rye', 'sal', 'sap', 'sar', 'saw', 'sbe', 'sbl', 'scc', 'scp', 'scr', 'sct', 'sea', 'see', 'sfn', 'sh', 'sha', 'she', 'sio', 'ski', 'so', 'sob', 'sol', 'son', 'soy', 'sp', 'spi', 'sr', 'sri', 'srs', 'ss', 'sss', 'st', 'stg', 'sua', 'sud', 'sun', 'sup', 'sur', 'svt', 'sw', 'sx', 'syn', 'syr', 'tab', 'tap', 'tar', 'tb', 'tbd', 'td', 'te', 'tea', 'tee', 'ten', 'th', 'tho', 'thr', 'ti', 'tia', 'tid', 'til', 'tka', 'tm', 'tn', 'toe', 'tos', 'tr', 'try', 'tss', 'tst', 'tts', 'tu', 'tum', 'tx', 'ua', 'uk', 'ul', 'unk', 'ups', 'uri', 'url', 'use', 'utc', 'uti', 'v', 'va', 'vac', 'vet', 'via', 'vid', 'vio', 'vit', 'vk', 'von', 'vp', 'vs', 'vt', 'w', 'war', 'was', 'way', 'wbc', 'well', 'will', 'wit', 'with', 'wu', 'www', 'x', 'xl', 'yes', 'yo', 'you', 'yrs', 'z']

df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [word for word in x if word not in meaningless])

In [34]:
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
allwords_list = list(allwords)
ans = sorted(allwords_list)
print(ans)
len(ans)

['aasthma', 'abation', 'abcess', 'abdomen', 'abdomil', 'abductus', 'ability', 'ablation', 'abnl', 'abnormal', 'abnormalities', 'abrasion', 'abrasions', 'abscess', 'absence', 'absolute', 'absorption', 'abuse', 'abx', 'access', 'accident', 'accidents', 'accute', 'acdf', 'acei', 'acetaminophen', 'achalasia', 'ache', 'aches', 'achilles', 'achiness', 'acidosis', 'acin', 'acitretin', 'acrylic', 'actenol', 'actifed', 'activation', 'activities', 'activity', 'actonel', 'actos', 'acute', 'acyclovir', 'adalimumab', 'adderall', 'addiction', 'addison', 'additiol', 'addition', 'additives', 'adductor', 'adenocarcinoma', 'adenomas', 'adequate', 'adhd', 'adhesion', 'adhesions', 'adhesives', 'adipex', 'adiposity', 'adjustment', 'admin', 'admission', 'admit', 'admitting', 'adpoa', 'adrel', 'adreline', 'adrenergic', 'adu', 'adult', 'advair', 'advance', 'adverse', 'advil', 'aeds', 'aemia', 'aerd', 'aeroallergens', 'affect', 'affects', 'afib', 'afloid', 'afluria', 'afranil', 'aftercare', 'afterwards', 'agav

5840

In [35]:
df.head()

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT,HEALTH_CONDITION
0,916600,33.0,F,,,pcn and bee venom,MODERNA,"[Dysphagia, Epiglottitis]",20-39 yr,"[bee, venom]"
1,916601,73.0,F,patient residing at nursing facility. see pati...,patient residing at nursing facility. see pati...,"""dairy""",MODERNA,"[Anxiety, Dyspnoea]",over_60 yr,"[facility, patients, nursing, facility, patients]"
2,916602,23.0,F,,,shellfish,PFIZER,"[Chest discomfort, Dysphagia, Pain in extremit...",20-39 yr,[]
3,916603,58.0,F,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","diclofec, novacaine, lidocaine, pickles, tomat...",MODERNA,"[Dizziness, Fatigue, Mobility decreased]",40-59 yr,"[kidney, infection, diverculitis, valve, prola..."
4,916604,47.0,F,,,,MODERNA,"[Injection site erythema, Injection site pruri...",40-59 yr,[]


## SAVE DF 


df.to_pickle('.spellcorrected_df_4_13.pkl')





In [36]:
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

5840

In [37]:
def get_wordnet_pos(tag):
    x, y = tag[0]
    if y.startswith('J'):
        return wordnet.ADJ
    elif y.startswith('V'):
        return wordnet.VERB
    elif y.startswith('N'):
        return wordnet.NOUN
    elif y.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [38]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from nltk import pos_tag
st = PorterStemmer()
wnl = WordNetLemmatizer()

In [39]:
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [wnl.lemmatize(w, pos=get_wordnet_pos(pos_tag([w]))) for w in x])


allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

5371

In [40]:
df.head()

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT,HEALTH_CONDITION
0,916600,33.0,F,,,pcn and bee venom,MODERNA,"[Dysphagia, Epiglottitis]",20-39 yr,"[bee, venom]"
1,916601,73.0,F,patient residing at nursing facility. see pati...,patient residing at nursing facility. see pati...,"""dairy""",MODERNA,"[Anxiety, Dyspnoea]",over_60 yr,"[facility, patient, nursing, facility, patient]"
2,916602,23.0,F,,,shellfish,PFIZER,"[Chest discomfort, Dysphagia, Pain in extremit...",20-39 yr,[]
3,916603,58.0,F,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","diclofec, novacaine, lidocaine, pickles, tomat...",MODERNA,"[Dizziness, Fatigue, Mobility decreased]",40-59 yr,"[kidney, infection, diverculitis, valve, prola..."
4,916604,47.0,F,,,,MODERNA,"[Injection site erythema, Injection site pruri...",40-59 yr,[]


In [41]:
# remove row with empty health_condition
#df_training=df[df['HEALTH_CONDITION'].apply(lambda x : len(x) >0)]
# df_training=df_training.copy()
df_training = df
# convert health condition list back to string
df_training['HEALTH_CONDITION'] = df_training['HEALTH_CONDITION'].apply(
    lambda x: " ".join(x))
df_training.head()

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT,HEALTH_CONDITION
0,916600,33.0,F,,,pcn and bee venom,MODERNA,"[Dysphagia, Epiglottitis]",20-39 yr,bee venom
1,916601,73.0,F,patient residing at nursing facility. see pati...,patient residing at nursing facility. see pati...,"""dairy""",MODERNA,"[Anxiety, Dyspnoea]",over_60 yr,facility patient nursing facility patient
2,916602,23.0,F,,,shellfish,PFIZER,"[Chest discomfort, Dysphagia, Pain in extremit...",20-39 yr,
3,916603,58.0,F,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","diclofec, novacaine, lidocaine, pickles, tomat...",MODERNA,"[Dizziness, Fatigue, Mobility decreased]",40-59 yr,kidney infection diverculitis valve prolapse o...
4,916604,47.0,F,,,,MODERNA,"[Injection site erythema, Injection site pruri...",40-59 yr,


# Trainning part

In [42]:
MODERNA = df_training[df_training['VAX_NAME'] == 'MODERNA']
PFIZER = df_training[df_training['VAX_NAME'] == 'PFIZER']

## Create training and test

In [43]:
from sklearn.model_selection import train_test_split
MODERNA_train, MODERNA_test = train_test_split(
    MODERNA, stratify=MODERNA['AGE_CAT'], test_size=0.05)
PFIZER_train, PFIZER_test = train_test_split(
    PFIZER, stratify=PFIZER['AGE_CAT'], test_size=0.05)

### Vectorize parameter for Moderna and Pfizer Kmeans model

In [44]:
# make df_training data to sparse matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
# define stop words
stop_w = text.ENGLISH_STOP_WORDS


def vectorizing(data_train):
    tfidf_vectorizer = TfidfVectorizer(stop_words=stop_w)
    ohe = OneHotEncoder(handle_unknown='ignore')

    tfidfs = tfidf_vectorizer.fit_transform(data_train['HEALTH_CONDITION'])
    tfidfs = pd.DataFrame(
        tfidfs.toarray(), columns=tfidf_vectorizer.get_feature_names())

    dummie_age_sex = ohe.fit_transform(data_train[['SEX', 'AGE_CAT']])
    dummie_age_sex = pd.DataFrame(
        dummie_age_sex.toarray(), columns=ohe.get_feature_names())
    vector = pd.concat([dummie_age_sex, tfidfs], axis=1)
    return {'vector': vector, 'tfidf': tfidf_vectorizer, "ohe": ohe}

In [45]:
moderna_vectorizer = vectorizing(MODERNA_train)
pfizer_vectorizer = vectorizing(PFIZER_train)

## Moderna K-means model training

In [78]:
from sklearn.cluster import KMeans
N_CLUSTER = 100
kmeans_MODERNA = KMeans(n_clusters=N_CLUSTER)
kmeans_MODERNA.fit(moderna_vectorizer.get('vector'))
MODERNA_train = MODERNA_train.copy()
MODERNA_train['CLUSTER'] = kmeans_MODERNA.labels_

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=40)
kmeans.fit(moderna_vector)

## Pfizer K-means model training¶

In [79]:
kmeans_PFIZER = KMeans(n_clusters=N_CLUSTER)
kmeans_PFIZER.fit(pfizer_vectorizer.get('vector'))
PFIZER_train = PFIZER_train.copy()
PFIZER_train['CLUSTER'] = kmeans_PFIZER.labels_

## How many instance belong to each cluster in Moderna

In [80]:
moderna_group = MODERNA_train.groupby(
    by='CLUSTER')['VAERS_ID'].agg('count').sort_values().to_frame()
moderna_group.columns = ['num of sample']
moderna_group

,num of sample
CLUSTER,
82,1
3,1
79,2
68,2
43,3
19,3
96,3
4,4
36,4


## what are in the cluster 

In [81]:
sym_freq = MODERNA_train[MODERNA_train['CLUSTER'] == 15]
sym_freq.head(10)

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT,HEALTH_CONDITION,CLUSTER
662,917328,21.0,F,,asthma,,MODERNA,"[Injection site erythema, Injection site pruri...",20-39 yr,aasthma,15
7047,924695,35.0,F,,asthma,"allergic to nuts, tea ,ibuprofen",MODERNA,"[Headache, Myalgia, Pain, Peripheral swelling,...",20-39 yr,aasthma alergies ibuprofen,15
4159,921473,34.0,F,,asthma,seasol and percocet.,MODERNA,[Dizziness],20-39 yr,aasthma seasol percocet,15
26601,1047777,36.0,F,,"asthma (allergy induced, well controlled), anx...",", other than pollen seasolly",MODERNA,"[Activated partial thromboplastin time, Antinu...",20-39 yr,aasthma alergies pain pollen,15
11055,928225,39.0,F,,asthma,,MODERNA,"[Injection site pruritus, Injection site rash]",20-39 yr,aasthma,15
22001,1010062,32.0,F,,"asthma, basic sal allergies",sensitive to doxycycline,MODERNA,"[Injection site discolouration, Injection site...",20-39 yr,aasthma alergies doxycycline,15
13384,930852,32.0,F,,asthma,,MODERNA,[Hypoaesthesia],20-39 yr,aasthma,15
921,917613,31.0,F,,"asthma, adhd",no,MODERNA,"[Chills, Fatigue, Injection site pain, Pain]",20-39 yr,aasthma adhd,15
14360,931392,26.0,F,,asthma,nickel allergy,MODERNA,"[Dizziness, Musculoskeletal stiffness, Pain, P...",20-39 yr,aasthma nickel alergies,15
16297,932846,26.0,F,,asthma,,MODERNA,"[Back pain, Dysstasia, Gait disturbance, Hypoa...",20-39 yr,aasthma,15


## Symptoms frequency in the cluster 

In [82]:
from itertools import chain
from collections import Counter
modern_group_sym_fre = pd.Series(Counter(chain.from_iterable(
    sym_freq.SYMPTOMS.dropna()))).sort_values(ascending=False).head(15).to_frame()
modern_group_sym_fre.columns = ['Frequency']
modern_group_sym_fre

,Frequency
Headache,18
Injection site pain,16
Nausea,14
Pain in extremity,13
Chills,12
Injection site erythema,11
Pain,11
Injection site pruritus,10
Pyrexia,10
Injection site swelling,9


## ELBOW METHOD

k = [x for x in range(5,50) if x%5==0]


wss=[]
for i in k:
    print(i)
    kmeans=KMeans(n_clusters=i, init='k-means++')
    kmeans=kmeans.fit(moderna_vector)
    wss_iter=kmeans.inertia_
    wss.append(wss_iter)

elbow=pd.DataFrame({'#Cluster':k , 'WSS': wss})

elbow

import seaborn as sns 
from matplotlib import pyplot as plt
sns.scatterplot(x=elbow['#Cluster'], y=elbow['WSS'], marker="+")

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_components=pca.fit_transform(moderna_vector)
pca_components=pd.DataFrame(pca_components, columns =['component1','component2'])
pca_components['cluster']=kmeans.labels_

pca_components


fig, ax = plt.subplots(figsize=(20,20))
sns.scatterplot(x=pca_components['component1'],y=pca_components['component2'], hue= pca_components['cluster'],ax=ax)


## Pftzer : How many instances belong to each cluster

In [83]:
PFIZER_train.groupby(by='CLUSTER')['VAERS_ID'].agg('count').sort_values()

CLUSTER
7        1
73       1
94       2
74       2
40       4
37       5
69       6
11       7
77       8
85       9
14       9
35       9
46      11
81      12
76      12
70      12
56      13
83      14
32      16
9       16
90      17
96      17
67      17
31      18
72      18
82      18
50      19
61      20
27      21
80      21
66      21
79      22
78      23
95      23
55      24
71      25
16      28
84      28
54      31
43      31
33      33
38      33
92      33
48      36
75      36
22      36
62      41
42      42
86      42
36      46
93      46
68      46
23      48
41      49
58      49
52      49
53      50
44      51
17      54
99      56
25      57
29      59
65      60
26      61
97      61
88      63
64      64
49      69
13      69
59      71
20      73
39      75
34      78
89      80
28      80
45      89
19      89
60      90
30      94
63      96
47     101
0      106
57     106
3      109
21     114
91     143
51     149
87     157
98     161
24     163
18

##  cluster detail

In [111]:
sym_freq = PFIZER_train[PFIZER_train['CLUSTER'] == 49]
sym_freq.head()

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT,HEALTH_CONDITION,CLUSTER
8213,925902,41.0,F,,asthma,,PFIZER,"[Chills, Pyrexia]",40-59 yr,aasthma,49
30715,1068818,44.0,F,,asthma,pcn,PFIZER,"[Asthenia, Burning sensation, Dyspnoea, Pyrexi...",40-59 yr,aasthma pcn,49
16690,933285,44.0,F,,asthma,,PFIZER,"[Body temperature, Dyspnoea, Fatigue, Investig...",40-59 yr,aasthma,49
30421,1068058,50.0,F,,asthma; covid-19 (if covid prior vaccition: yes),,PFIZER,"[Chills, Headache, Neck pain, Pain, Sinusitis]",40-59 yr,aasthma covid vaccition,49
29491,1061967,59.0,F,,asthma (chronic asthma but controlled),,PFIZER,"[Fatigue, Lymph node pain, Lymphadenopathy]",40-59 yr,aasthma aasthma,49


### Symptoms frequency in the cluster

In [112]:
from itertools import chain
from collections import Counter
pd.Series(Counter(chain.from_iterable(sym_freq.SYMPTOMS.dropna()))
          ).sort_values(ascending=False).head(10).to_frame()

Headache                        23
Chills                          16
Pyrexia                         16
Fatigue                         14
Pain                            12
Pain in extremity               11
Nausea                           9
Dyspnoea                         8
Pruritus                         7
Arthralgia                       7
Dizziness                        6
Myalgia                          6
Cough                            5
Back pain                        5
Lymphadenopathy                  5
Heart rate increased             5
Diarrhoea                        5
Injection site pain              5
Body temperature                 5
Rash                             5
Chest discomfort                 4
SARS-CoV-2 test                  4
Paraesthesia oral                3
Paraesthesia                     3
Respiratory tract congestion     3
SARS-CoV-2 test negative         3
Heart rate                       3
SARS-CoV-2 test positive         3
Asthenia            

### user input 

In [86]:
# a New sample
letter = "SHELLFISH"
age = 25
sex = 'U'
agecat = convertAge(age)
new_sample = pd.DataFrame(
    {'sex': [sex], 'age_cat': [agecat]}, columns=['age_cat', 'sex'])

### Vectorize user input for Moderna & Pfizer kmeans prediction

In [87]:
def get_prediction(new_sample, vectorizer, model):
    # one hot encode [sex, age_cat]
    sample_dummi = vectorizer.get("ohe").transform(
        new_sample[['SEX', 'AGE_CAT']])
    sample_dummi = pd.DataFrame(sample_dummi.toarray(
    ), columns=vectorizer.get("ohe").get_feature_names())

    # vectorize the new sample

    sample_tfidf = vectorizer.get("tfidf").transform(
        new_sample['HEALTH_CONDITION'])
    sample_tfidf = pd.DataFrame(sample_tfidf.toarray(
    ), columns=vectorizer.get("tfidf").get_feature_names())

    # combine tfidf and dummie to one df and use it as a parameter for the kmeans prediction
    sampleData = pd.concat([sample_dummi, sample_tfidf], axis=1)

    prediction = model.predict(sampleData)
    return prediction

In [88]:
def get_top10_sym_per_cluster(data_train, number_cluster):
    data_by_cluster = {}
    for i in range(N_CLUSTER):
        cluster = data_train[data_train['CLUSTER'] == i]
        cluster = pd.Series(Counter(chain.from_iterable(
            cluster.SYMPTOMS.dropna()))).sort_values(ascending=False).head(10).to_frame()
        cluster.reset_index()
        cluster.columns = ['qty']
        data_by_cluster[i] = cluster
    return data_by_cluster


def get_symptom_accuracy_count(data_test, top_10symp_by_cluster):
    MATCHING = []
    TOP10=[]
    for index, row in data_test.iterrows():
        count = 0
        for word in row['SYMPTOMS']:
            if word in top_10symp_by_cluster.get(row.PREDICTION).index.tolist():
                count = count+1
        MATCHING.append(count/len(row['SYMPTOMS']))
        TOP10.append(top_10symp_by_cluster.get(row.PREDICTION).index.tolist())
    data_test['MATCHING'] = MATCHING
    data_test['TOP10_SYMP']=TOP10
    return data_test

In [89]:
# make a copy of the test sets
MODERNA_test = MODERNA_test.copy()
PFIZER_test = PFIZER_test.copy()

# get cluster labels for the test sets
MODERNA_test['PREDICTION'] = get_prediction(
    MODERNA_test, moderna_vectorizer, kmeans_MODERNA)
PFIZER_test['PREDICTION'] = get_prediction(
    PFIZER_test, pfizer_vectorizer, kmeans_PFIZER)

# get the 10 symptoms of each cluster in the training sets and store them in dictionaries
top10_symp_moderna = get_top10_sym_per_cluster(MODERNA_train, N_CLUSTER)
top10_symp_pfizer = get_top10_sym_per_cluster(PFIZER_train, N_CLUSTER)

# for each instance in the test set, find out how many of its symptoms are matching with the top 10 symptoms of the same cluster in the training set
MODERNA_test = get_symptom_accuracy_count(MODERNA_test, top10_symp_moderna)
PFIZER_test = get_symptom_accuracy_count(PFIZER_test, top10_symp_pfizer)

In [90]:
top10_symp_moderna

{0:                          qty
 Headache                 749
 Pyrexia                  505
 Chills                   491
 Pain                     470
 Injection site erythema  464
 Fatigue                  460
 Nausea                   423
 Injection site pain      408
 Myalgia                  344
 Dizziness                341,
 1:           qty
 Death     255
 Pyrexia   189
 Chills    141
 Fatigue   141
 Headache  134
 Myalgia   114
 Asthenia  100
 Nausea     97
 Dyspnoea   85
 Pain       82,
 2:                          qty
 Headache                 581
 Pyrexia                  465
 Injection site erythema  459
 Chills                   437
 Nausea                   425
 Fatigue                  418
 Pain                     411
 Injection site pain      399
 Pain in extremity        358
 Injection site swelling  356,
 3:           qty
 Fatigue     1
 Headache    1
 Pain        1,
 4:                             qty
 Anxiety                       1
 Pain                         

In [91]:
MODERNA_test[['AGE_CAT','SEX','VAX_NAME','AGE_CAT','HEALTH_CONDITION','SYMPTOMS','PREDICTION','MATCHING','TOP10_SYMP']].head(10)

,AGE_CAT,SEX,VAX_NAME,AGE_CAT,HEALTH_CONDITION,SYMPTOMS,PREDICTION,MATCHING,TOP10_SYMP
27522,over_60 yr,F,MODERNA,over_60 yr,iodine,"[Dizziness, Full blood count, Headache, Metabo...",7,0.250000,"[Headache, Pyrexia, Fatigue, Injection site er..."
9830,40-59 yr,M,MODERNA,40-59 yr,,"[Injection site pain, Myalgia]",5,1.000000,"[Headache, Pyrexia, Chills, Fatigue, Myalgia, ..."
258,40-59 yr,F,MODERNA,40-59 yr,,"[Injection site pain, Pain in extremity, Rash ...",0,0.333333,"[Headache, Pyrexia, Chills, Pain, Injection si..."
25875,over_60 yr,F,MODERNA,over_60 yr,,"[Asthenia, Balance disorder, Dizziness, Dyssta...",7,0.000000,"[Headache, Pyrexia, Fatigue, Injection site er..."
2218,20-39 yr,F,MODERNA,20-39 yr,codeine latex,"[Axillary pain, Injection site erythema, Injec...",8,0.200000,"[Pain in extremity, Dizziness, Headache, Chill..."
12979,over_60 yr,F,MODERNA,over_60 yr,chloesterol sulfur,"[Burning sensation, Rash]",89,0.000000,"[Injection site erythema, Dizziness, Injection..."
8453,20-39 yr,F,MODERNA,20-39 yr,seasol alergies,"[Injection site erythema, Injection site rash,...",55,0.200000,"[Headache, Pain in extremity, Injection site e..."
12433,40-59 yr,F,MODERNA,40-59 yr,,[Unevaluable event],0,0.000000,"[Headache, Pyrexia, Chills, Pain, Injection si..."
23024,over_60 yr,F,MODERNA,over_60 yr,diabertes hyperlidipdemia hypertenision,"[Echocardiogram, Facial paralysis, Magnetic re...",44,0.000000,"[Pyrexia, Death, Pain, Fatigue, Injection site..."
9524,40-59 yr,F,MODERNA,40-59 yr,blood pressure kiwi penicillin,"[Headache, Migraine, Pain in extremity, Vaccin...",12,0.333333,"[Pain, Headache, Fatigue, Injection site swell..."


In [92]:
PFIZER_test[['AGE_CAT','SEX','VAX_NAME','AGE_CAT','HEALTH_CONDITION','SYMPTOMS','PREDICTION','MATCHING','TOP10_SYMP']].head(10)

,AGE_CAT,SEX,VAX_NAME,AGE_CAT,HEALTH_CONDITION,SYMPTOMS,PREDICTION,MATCHING,TOP10_SYMP
22899,over_60 yr,F,PFIZER,over_60 yr,arteriole blood pressure chloesterol colities ...,"[Asthenia, Back pain, Balance disorder, Blood ...",51,0.222222,"[Pain in extremity, Dizziness, Headache, Fatig..."
5537,40-59 yr,F,PFIZER,40-59 yr,dystrophy dystrophy tetracycline strawberry ch...,"[Asthenia, Blood test, Dizziness, Dysphonia, E...",4,0.181818,"[Headache, Pyrexia, Pain, Chills, Fatigue, Nau..."
1779,20-39 yr,F,PFIZER,20-39 yr,,[Heart rate increased],1,0.000000,"[Headache, Chills, Pyrexia, Fatigue, Pain, Nau..."
4546,over_60 yr,M,PFIZER,over_60 yr,,[Fatigue],5,1.000000,"[Death, Pyrexia, Dyspnoea, Fatigue, Headache, ..."
21704,20-39 yr,M,PFIZER,20-39 yr,,[Fatigue],15,1.000000,"[Headache, Pyrexia, Chills, Pain, Fatigue, Mya..."
29045,unknown_age,F,PFIZER,unknown_age,,"[Arthralgia, Body temperature, Chills, Cough, ...",8,0.500000,"[SARS-CoV-2 test positive, COVID-19, Headache,..."
10644,over_60 yr,F,PFIZER,over_60 yr,blood pressure verbatim blood pressure chloest...,"[Pain in extremity, Rash, Rash pruritic]",51,0.333333,"[Pain in extremity, Dizziness, Headache, Fatig..."
6447,40-59 yr,F,PFIZER,40-59 yr,codeine food gluten dairy sensitivity,"[Fatigue, Headache, Myalgia, Oropharyngeal pain]",79,0.500000,"[Fatigue, Headache, Nausea, Pain, Chills, Feel..."
32144,unknown_age,U,PFIZER,unknown_age,,"[COVID-19, Drug ineffective]",6,1.000000,"[SARS-CoV-2 test positive, COVID-19, Headache,..."
20412,40-59 yr,F,PFIZER,40-59 yr,borderline hypertenision hive wheal drug perim...,"[Drug interaction, Dyspnoea, Fatigue, Inflamma...",4,0.200000,"[Headache, Pyrexia, Pain, Chills, Fatigue, Nau..."


In [93]:
print(MODERNA_test['MATCHING'].mean())
print(PFIZER_test['MATCHING'].mean())

0.32776672608754526
0.3119108536458563


In [94]:
MODERNA_test

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT,HEALTH_CONDITION,PREDICTION,MATCHING,TOP10_SYMP
27522,1049583,66.0,F,,,latex iodine,MODERNA,"[Dizziness, Full blood count, Headache, Metabo...",over_60 yr,iodine,7,0.250000,"[Headache, Pyrexia, Fatigue, Injection site er..."
9830,937955,56.0,M,,,,MODERNA,"[Injection site pain, Myalgia]",40-59 yr,,5,1.000000,"[Headache, Pyrexia, Chills, Fatigue, Myalgia, ..."
258,916888,50.0,F,,,,MODERNA,"[Injection site pain, Pain in extremity, Rash ...",40-59 yr,,0,0.333333,"[Headache, Pyrexia, Chills, Pain, Injection si..."
25875,1047027,73.0,F,,,,MODERNA,"[Asthenia, Balance disorder, Dizziness, Dyssta...",over_60 yr,,7,0.000000,"[Headache, Pyrexia, Fatigue, Injection site er..."
2218,919157,35.0,F,reported,,"penicillin, codeine, latex",MODERNA,"[Axillary pain, Injection site erythema, Injec...",20-39 yr,codeine latex,8,0.200000,"[Pain in extremity, Dizziness, Headache, Chill..."
12979,930406,67.0,F,no,high bp. high cholesterol,sulfur,MODERNA,"[Burning sensation, Rash]",over_60 yr,chloesterol sulfur,89,0.000000,"[Injection site erythema, Dizziness, Injection..."
8453,926157,34.0,F,,seasol allergies,,MODERNA,"[Injection site erythema, Injection site rash,...",20-39 yr,seasol alergies,55,0.200000,"[Headache, Pain in extremity, Injection site e..."
12433,929812,51.0,F,,,,MODERNA,[Unevaluable event],40-59 yr,,0,0.000000,"[Headache, Pyrexia, Chills, Pain, Injection si..."
23024,1022613,85.0,F,,"diabetes, osteoarthritis, hyperlipidemia, hype...",,MODERNA,"[Echocardiogram, Facial paralysis, Magnetic re...",over_60 yr,diabertes hyperlidipdemia hypertenision,44,0.000000,"[Pyrexia, Death, Pain, Fatigue, Injection site..."
9524,927309,57.0,F,,high blood pressure,"kiwi, penicillin",MODERNA,"[Headache, Migraine, Pain in extremity, Vaccin...",40-59 yr,blood pressure kiwi penicillin,12,0.333333,"[Pain, Headache, Fatigue, Injection site swell..."


In [95]:
from joblib import dump
dump(kmeans_MODERNA, 'kmeans_MODERNA_model.joblib')
dump(kmeans_PFIZER, 'kmeans_PFIZER_model.joblib')
dump(moderna_vectorizer.get('ohe'), 'onehotendcoder_moderna.joblib')
dump(pfizer_vectorizer.get("ohe"), 'onehotendcoder_pfizer.joblib')
dump(moderna_vectorizer.get('tfidf'), 'tfidf_vectorizer_moderna.joblib')
dump(pfizer_vectorizer.get("tfidf"), 'tfidf_vectorizer_pfizer.joblib')

['tfidf_vectorizer_pfizer.joblib']

In [96]:
MODERNA_train.to_pickle("MODERNA_DF.pkl")
PFIZER_train.to_pickle("PFIZER_DF.pkl")

In [97]:
df_Moderna = pd.read_pickle("MODERNA_DF.pkl")
df_Pfizer = pd.read_pickle("PFIZER_DF.pkl")

In [98]:
import joblib
MODERNA_model = joblib.load('kmeans_MODERNA_model.joblib')
PFIZER_model = joblib.load('kmeans_PFIZER_model.joblib')
onehot_moderna = joblib.load('onehotendcoder_moderna.joblib')
onehot_pfizer = joblib.load('onehotendcoder_pfizer.joblib')
tfidf_enc_moderna = joblib.load('tfidf_vectorizer_moderna.joblib')
tfidf_enc_pfizer = joblib.load('tfidf_vectorizer_pfizer.joblib')

## Testing after loaded the saved models using joblib

In [99]:
# one hot encode using Moderna OHE
test_sample_dummi_MODERNA = onehot_moderna.transform(
    new_sample[['sex', 'age_cat']])
test_sample_dummi_MODERNA = pd.DataFrame(
    test_sample_dummi_MODERNA.toarray(), columns=onehot_moderna.get_feature_names())

# vectorize the new sample

test_sample_tfidf_MODERNA = tfidf_enc_moderna.transform([letter])
test_sample_tfidf_MODERNA = pd.DataFrame(
    test_sample_tfidf_MODERNA.toarray(), columns=tfidf_enc_moderna.get_feature_names())

# combine tfidf and dummie to one df and use it as a parameter for the kmeans prediction
test_sampleData_MODERNA = pd.concat(
    [test_sample_dummi_MODERNA, test_sample_tfidf_MODERNA], axis=1)
MODERNA_model.predict(test_sampleData_MODERNA)

array([35])

In [100]:
# one hot encode using Moderna OHE
test_sample_dummi_PFIZER = onehot_pfizer.transform(
    new_sample[['sex', 'age_cat']])
test_sample_dummi_PFIZER = pd.DataFrame(
    test_sample_dummi_PFIZER.toarray(), columns=onehot_pfizer.get_feature_names())

# vectorize the new sample

test_sample_tfidf_PFIZER = tfidf_enc_pfizer.transform([letter])
test_sample_tfidf_PFIZER = pd.DataFrame(
    test_sample_tfidf_PFIZER.toarray(), columns=tfidf_enc_pfizer.get_feature_names())

# combine tfidf and dummie to one df and use it as a parameter for the kmeans prediction
test_sampleData_PFIZER = pd.concat(
    [test_sample_dummi_PFIZER, test_sample_tfidf_PFIZER], axis=1)
PFIZER_model.predict(test_sampleData_PFIZER)

array([27])

In [101]:
pd.Series(Counter(chain.from_iterable(MODERNA['SYMPTOMS'].dropna()))).sort_values(
    ascending=False).head(30)

Headache                   3227
Pyrexia                    2738
Chills                     2435
Fatigue                    2393
Pain                       2183
Nausea                     2006
Injection site erythema    1983
Injection site pain        1871
Pain in extremity          1604
Dizziness                  1566
Injection site swelling    1433
Myalgia                    1426
Injection site pruritus    1284
Injection site warmth      1053
Pruritus                    997
Arthralgia                  970
Dyspnoea                    928
Vomiting                    915
Erythema                    886
Death                       852
Rash                        821
Asthenia                    685
Urticaria                   628
Diarrhoea                   571
Injection site rash         533
Paraesthesia                501
Lymphadenopathy             482
Malaise                     450
Peripheral swelling         438
Hypoaesthesia               422
dtype: int64

In [102]:
dd = pd.Series(Counter(chain.from_iterable(PFIZER.SYMPTOMS.dropna()))
               ).sort_values(ascending=False).head(10).to_frame()
dd.columns = ["PFIZER"]
dd

,PFIZER
Headache,3501
Pyrexia,2607
Chills,2575
Fatigue,2522
Pain,2290
Nausea,1906
Dizziness,1744
Pain in extremity,1422
Myalgia,1290
Dyspnoea,1116


In [103]:
MO = MODERNA['AGE_CAT'].value_counts().to_frame()
MO.columns = ["MODERNA"]
PF = PFIZER['AGE_CAT'].value_counts().to_frame()
PF.columns = ["PFIZER"]
#pd.options.display.float_format = '{:,.0f}'.format
dd = pd.concat([MO, PF], axis=1).fillna(0)
dd.loc['Total', :] = dd.sum(axis=0)

In [104]:
dd

,MODERNA,PFIZER
40-59 yr,5712.0,4991.0
over_60 yr,5126.0,4685.0
20-39 yr,4533.0,4295.0
unknown_age,798.0,2225.0
13-19 yr,194.0,73.0
0-2 yr,21.0,16.0
3-12 yr,2.0,0.0
Total,16386.0,16285.0


In [105]:
MO_de_relate = MODERNA[MODERNA['SYMPTOMS'].apply(lambda x: 'Death' in x)].groupby(
    by=['AGE_CAT'])['VAERS_ID'].count().to_frame()
MO_de_relate.columns = ['MODERNA(#death)']

In [106]:
Pf_de_relate = PFIZER[PFIZER['SYMPTOMS'].apply(lambda x: 'Death' in x)].groupby(
    by=['AGE_CAT'])['VAERS_ID'].count().to_frame()
Pf_de_relate.columns = ['PFIZER(#death)']
Pf_de_relate.sort_values(by='PFIZER(#death)', ascending=False)
de = pd.concat([MO_de_relate, Pf_de_relate], axis=1).fillna(0)
de.loc['Total', :] = de.sum(axis=0)
summ = pd.concat([dd, de], axis=1).fillna(0)

summ['%'] = (summ['MODERNA(#death)']+summ['PFIZER(#death)'])/(summ['MODERNA']+summ['PFIZER'])*100
summ

,MODERNA,PFIZER,MODERNA(#death),PFIZER(#death),%
40-59 yr,5712.0,4991.0,62.0,52.0,1.065122
over_60 yr,5126.0,4685.0,746.0,617.0,13.892570
20-39 yr,4533.0,4295.0,6.0,14.0,0.226552
unknown_age,798.0,2225.0,36.0,56.0,3.043334
13-19 yr,194.0,73.0,2.0,0.0,0.749064
0-2 yr,21.0,16.0,0.0,2.0,5.405405
3-12 yr,2.0,0.0,0.0,0.0,0.000000
Total,16386.0,16285.0,852.0,741.0,4.875884


In [107]:
MO = MODERNA['SEX'].value_counts().to_frame()
MO.columns = ["MODERNA"]
PF = PFIZER['SEX'].value_counts().to_frame()
PF.columns = ["PFIZER"]
#pd.options.display.float_format = '{:,.0f}'.format
dd = pd.concat([MO, PF], axis=1).fillna(0)
dd.loc['Total', :] = dd.sum(axis=0)

In [108]:
dd

,MODERNA,PFIZER
F,11926.0,11628.0
M,4213.0,4083.0
U,247.0,574.0
Total,16386.0,16285.0


In [109]:
MO_de_relate = MODERNA[MODERNA['SYMPTOMS'].apply(lambda x: 'Death' in x)].groupby(
    by=['SEX'])['VAERS_ID'].count().to_frame()
MO_de_relate.columns = ['MODERNA(#death)']

Pf_de_relate = PFIZER[PFIZER['SYMPTOMS'].apply(lambda x: 'Death' in x)].groupby(
    by=['SEX'])['VAERS_ID'].count().to_frame()
Pf_de_relate.columns = ['PFIZER(#death)']
Pf_de_relate.sort_values(by='PFIZER(#death)', ascending=False)
de = pd.concat([MO_de_relate, Pf_de_relate], axis=1).fillna(0)
de.loc['Total', :] = de.sum(axis=0)
summ = pd.concat([dd, de], axis=1).fillna(0)
summ['%'] = (summ['MODERNA(#death)']+summ['PFIZER(#death)'])/(summ['MODERNA']+summ['PFIZER'])*100
summ

,MODERNA,PFIZER,MODERNA(#death),PFIZER(#death),%
F,11926.0,11628.0,356.0,351.0,3.001613
M,4213.0,4083.0,486.0,364.0,10.245902
U,247.0,574.0,10.0,26.0,4.384896
Total,16386.0,16285.0,852.0,741.0,4.875884
